<a href="https://colab.research.google.com/github/TanmayPK/RAG-Chatbot/blob/main/PDF_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pypdf
!pip install tiktoken
!pip install openai
!pip install cohere
!pip install llama-index
!pip install accelerate

In [2]:
!pip install torch sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.3 MB/s eta 0:00:00


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os

In [6]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["Speech-Science-2-annotated-final.pdf"]
).load_data()

In [7]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

24 

<class 'llama_index.schema.Document'>
Doc ID: 9f82810f-7aa8-4d65-9a41-0f3ec07d1918
Text: Speech Science 2 Astrid van Wieringen 2: source -filter/F0 and
resonances/ segmental info vowels Logopedische en Audiologische
Wetenschappen KULeuven Experimental ORL 1


In [8]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [9]:
from llama_index.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [10]:
nodes = node_parser.get_nodes_from_documents([document])

In [11]:
from llama_index.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

o, ɔ, ɑ/
•Phonetic features : back tongue position, various degrees of tongue height
•Acoustic cues : small separation between F1 and F2, large separation between F2 and F3
•These sounds vary primarily in the frequency of F1 withvaries withvowel height
•The lower the tongue position, the higher the F1 frequency
•The phonemes /u, o, ɔ, ɑ/ have an inherently long duration , /ʊ / has short durationSummary of spectral and temporal properties of back vowels
Lass ,


In [12]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

range of plot isfrom 0-10000 HzSummary of spectral and temporal properties of front vowels
Lass , 1996; Kent & Read, 1992
Time (s)0 0.20104/i/ from 'piet'
Time (s)0 0.0294331-0.16420.259
0
f= 1/T

21•Phonemes /u, ʊ, o, ɔ, ɑ/
•Phonetic features : back tongue position, various degrees of tongue height
•Acoustic cues : small separation between F1 and F2, large separation between F2 and F3
•These sounds vary primarily in the frequency of F1 withvaries withvowel height
•The lower the tongue position, the higher the F1 frequency
•The phonemes /u, o, ɔ, ɑ/ have an inherently long duration , /ʊ / has short durationSummary of spectral and temporal properties of back vowels
Lass , 1996; Kent & Read, 1992
Time (s)0 0.20104/u/ from 'poet'
Time (s)0 0.0322222-0.28560.3335
0

22•Phonemes /ʌ, ə/
•Phonetic features : central tongue position
•Acoustic cues : a uniform formant pattern (formant frequencies tend to beequally
spaced )
•Duration : /ʌ / longer than /ə /Summary of spectral and temporal proper

In [13]:
from llama_index.llms import HuggingFaceLLM

In [14]:
llm = HuggingFaceLLM(model_name="mistralai/Mistral-7B-Instruct-v0.2", tokenizer_name = 'mistralai/Mistral-7B-Instruct-v0.2')

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [15]:
from llama_index import ServiceContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [16]:
from llama_index import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")

In [17]:
import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context
    )


In [18]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

retriever = AutoMergingRetriever(
    automerging_retriever,
    automerging_index.storage_context,
    verbose=True
)

rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank], service_context = auto_merging_context
)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [21]:
auto_merging_response = auto_merging_engine.query(
    "What are the spectral and temporal properties of back vowels ?"
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacty of 15.77 GiB of which 488.38 MiB is free. Process 8856 has 15.29 GiB memory in use. Of the allocated memory 14.74 GiB is allocated by PyTorch, and 195.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:
from llama_index.response.notebook_utils import display_response

display_response(auto_merging_response)

NameError: name 'auto_merging_response' is not defined

In [22]:
auto_merging_response = auto_merging_engine.query(
    "What are the spectral and temporal properties of front vowels ?"
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [23]:
display_response(auto_merging_response)

**`Final Response:`** Front vowels are phonemes with back tongue position and various degrees of tongue height. The acoustic cues for front vowels include a small separation between F1 and F2, and a large separation between F2 and F3. The sounds vary primarily in the frequency of F1, which is influenced by the vowel height. The lower the tongue position, the higher the F1 frequency. The front vowels /u, o, ɔ, ɑ/ have inherently long durations, while /ʊ/ has a short duration. The spectral properties include a formant pattern, spectrum (slope, rate-of-frequency change in spectrum), duration, fundamental frequency, formant bandwidth, and formant amplitude. The temporal properties include the duration and utterance position (syntactic feature), as well as word familiarity. The example given is from the Dutch language and the frequency range is from 0-10000 Hz.